In [57]:
def loadDate():
    return [[1,3,4],[2,3,5],[1,2,3,5],[2,5]]

def createC1(dataSet):
    C1 =[]#C1为所有大小为1的候选项集的集合
    for transactin in dataSet:
        for item in transactin:
            if not [item] in C1:
                C1.append([item])#注意这里不是只添加每一个物品项，而是整个物品项的集合
    C1.sort()#排序
    return map(frozenset,C1)#对C1的每一项构建一个不变的集合，在这里用了frozenset，因为后面的项集列表是按照一定的格式存放的，是不可改变的，之后将会将这些集合作为字典使用
                
def scanD(D,Ck,minSupport):#用于从C1到L1
    ssCnt ={}#创建空字典
    for tid in D:
        for can in Ck:#遍历所有的交易记录和候选集中的数据
            if can.issubset(tid):
                #if not ssCnt.has_key(can): ssCnt[can]=1  Python3中的这个has_key方法去掉了
                if can in ssCnt:ssCnt[can] += 1#如果C1的集合是记录的一部分，增加计数这里字典的键是集合
                else : ssCnt[can] = 1
    numItems = float(len(D))
    retList = []
    supportData = {}#包含满足最小支持度要求的集合
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)#在列表首部添加任意的新的集合
        supportData[key] = support
    return retList,supportData


#循环的取第i和i+1的前k-2项，如何相等就合并
def aprioriGen(Lk,k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1,lenLk):
            L1 = list(Lk[i])[:k-2];  L2 = list(Lk[j])[:k-2]#与下面的算法中的k=2保持一致
            L1.sort();L2.sort()
            if L1 == L2:
                retList.append(Lk[i]|Lk[j])
    return retList

def apriori(dataSet,minSupport=0.5):
    C1 = list(createC1(dataSet))
    D = list(map(set,dataSet))
    L1 ,supportData = scanD(D,C1,minSupport)
    L = [L1]
    k=2#跟下面的k-2保持一致
    while (len(L[k-2])>0):
        Ck = aprioriGen(L[k-2],k)
        Lk ,supK = scanD(D,Ck,minSupport)#这里会丢掉不满足最小支持度的项集
        supportData.update(supK)#包含所有项极其支持度
        L.append(Lk)
        k+=1
    return L,supportData


def generateRules(L, supportData, minConf=0.7):  #supportData是一个来自于scanD的字典
    bigRuleList = []
    for i in range(1, len(L)):#由于无法从单元素项集中构建关联规则，故从i=1开始
        for freqSet in L[i]:#如果L是{1,2,3}则H为{{1},{2},{3}
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):#如果频繁项集的元素超过两个，就用rulesFromConseq
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList         

def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #计算可信度H(HUP)/H(P)
        if conf >= minConf: 
            print(freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])#查看该频繁项集是否达到可以移除大小为m的子集
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#包含所有的规划
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #当频繁项集在元素大于2的时候继续进行迭代
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)
            
def pntRules(ruleList, itemMeaning):
    for ruleTup in ruleList:
        for item in ruleTup[0]:
            print(itemMeaning[item])
        print("           -------->")
        for item in ruleTup[1]:
            print(itemMeaning[item])
        print("confidence: %f" % ruleTup[2])
        print 

In [56]:
from time import sleep
from votesmart import votesmart
votesmart.apikey = 'a7fa40adec6f4a77178799fae4441030'
bills = votesmart.votes.getBillsByStateRecent()
for bill in bills:
    print(bill.title,bill.billId)

SyntaxError: invalid syntax (votesmart.py, line 219)

In [59]:
dataSet = loadDate()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [60]:
C1 = list(createC1(dataSet))
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [61]:
D = list(map(set,dataSet))
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [62]:
L1,supportData0 = scanD(D,C1,0.5)
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [63]:
supportData0

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75}

In [64]:
L,supportData = apriori(dataSet)

In [65]:
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [66]:
supportData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({1, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3}): 0.5,
 frozenset({1, 5}): 0.25,
 frozenset({1, 2}): 0.25,
 frozenset({2, 3, 5}): 0.5}

In [67]:
L[0]
L[1]
#L[2]

[frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})]

In [68]:
rules= generateRules(L,supportData,minConf=0.7)
rules

frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({1}) --> frozenset({3}) conf: 1.0


[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0)]

In [69]:
rules= generateRules(L,supportData,minConf=0.5)
rules

frozenset({3}) --> frozenset({2}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({5}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({3}) --> frozenset({1}) conf: 0.6666666666666666
frozenset({1}) --> frozenset({3}) conf: 1.0
frozenset({5}) --> frozenset({2, 3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({2, 5}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3, 5}) conf: 0.6666666666666666


[(frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]

In [71]:
mushDataSet = [line.split() for line in open('E:/谢远东/机器学习/机器学习实践/机器学习实战源代码/machinelearninginaction/Ch11/mushroom.dat').readlines()]
mushDataSet

[['1',
  '3',
  '9',
  '13',
  '23',
  '25',
  '34',
  '36',
  '38',
  '40',
  '52',
  '54',
  '59',
  '63',
  '67',
  '76',
  '85',
  '86',
  '90',
  '93',
  '98',
  '107',
  '113'],
 ['2',
  '3',
  '9',
  '14',
  '23',
  '26',
  '34',
  '36',
  '39',
  '40',
  '52',
  '55',
  '59',
  '63',
  '67',
  '76',
  '85',
  '86',
  '90',
  '93',
  '99',
  '108',
  '114'],
 ['2',
  '4',
  '9',
  '15',
  '23',
  '27',
  '34',
  '36',
  '39',
  '41',
  '52',
  '55',
  '59',
  '63',
  '67',
  '76',
  '85',
  '86',
  '90',
  '93',
  '99',
  '108',
  '115'],
 ['1',
  '3',
  '10',
  '15',
  '23',
  '25',
  '34',
  '36',
  '38',
  '41',
  '52',
  '54',
  '59',
  '63',
  '67',
  '76',
  '85',
  '86',
  '90',
  '93',
  '98',
  '107',
  '113'],
 ['2',
  '3',
  '9',
  '16',
  '24',
  '28',
  '34',
  '37',
  '39',
  '40',
  '53',
  '54',
  '59',
  '63',
  '67',
  '76',
  '85',
  '86',
  '90',
  '94',
  '99',
  '109',
  '114'],
 ['2',
  '3',
  '10',
  '14',
  '23',
  '26',
  '34',
  '36',
  '39',
  '41',
 

In [72]:
L,suppoData = apriori(mushDataSet,minSupport=0.3)


In [75]:
#如果某个样本有毒值为2
for i in range(len(L)):
    for item in L[i]:
        if item.intersection('2'): print(item)


frozenset({'2'})
frozenset({'2', '28'})
frozenset({'53', '2'})
frozenset({'2', '23'})
frozenset({'34', '2'})
frozenset({'36', '2'})
frozenset({'59', '2'})
frozenset({'2', '63'})
frozenset({'2', '67'})
frozenset({'2', '76'})
frozenset({'2', '85'})
frozenset({'86', '2'})
frozenset({'90', '2'})
frozenset({'2', '93'})
frozenset({'2', '39'})
frozenset({'53', '2', '28'})
frozenset({'53', '2', '39'})
frozenset({'53', '2', '90'})
frozenset({'53', '86', '2'})
frozenset({'53', '2', '85'})
frozenset({'34', '53', '2'})
frozenset({'2', '39', '28'})
frozenset({'2', '85', '28'})
frozenset({'2', '63', '28'})
frozenset({'90', '2', '28'})
frozenset({'86', '2', '28'})
frozenset({'59', '2', '28'})
frozenset({'34', '2', '28'})
frozenset({'2', '39', '93'})
frozenset({'90', '2', '93'})
frozenset({'90', '2', '86'})
frozenset({'90', '2', '85'})
frozenset({'90', '2', '63'})
frozenset({'90', '2', '23'})
frozenset({'90', '2', '39'})
frozenset({'86', '2', '93'})
frozenset({'86', '2', '85'})
frozenset({'86', '2', '